<a href="https://colab.research.google.com/github/eniangnsa/Movie-Recommendation-System/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


Data collection and preprocessing

In [2]:
data = pd.read_csv("movies.csv")

In [3]:
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
data.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [5]:
data.shape

(4803, 24)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

Selecting relevant features

In [7]:
data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [8]:
relevant_features = ['genres','keywords', 'tagline', 'director' ,'cast']

replace null values with null strings

In [9]:
for feature in relevant_features:
  data[feature] = data[feature].fillna('')

Combining all the 5 selected features

In [10]:
combined_features = data['genres']+' '+data['keywords']+' '+data['tagline']+' '+data['cast']+' '+data['director']

In [11]:
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

Convert the text data to vectors

In [12]:
vectorizer = TfidfVectorizer()

In [13]:
feature_vectors = vectorizer.fit_transform(combined_features)

Cosine Similarity

In [14]:
similarity = cosine_similarity(feature_vectors)

In [15]:
similarity

array([[1.        , 0.07219487, 0.037733  , ..., 0.        , 0.        ,
        0.        ],
       [0.07219487, 1.        , 0.03281499, ..., 0.03575545, 0.        ,
        0.        ],
       [0.037733  , 0.03281499, 1.        , ..., 0.        , 0.05389661,
        0.        ],
       ...,
       [0.        , 0.03575545, 0.        , ..., 1.        , 0.        ,
        0.02651502],
       [0.        , 0.        , 0.05389661, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02651502, 0.        ,
        1.        ]])

In [16]:
similarity.shape

(4803, 4803)

Getting the movie name from the user

In [17]:
movie_name = input('Enter your favourite name: ')

Enter your favourite name: Eyes in the sky


Create a list with all movies

In [18]:
names_of_movies = data['title'].tolist()

In [19]:
names_of_movies

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

Finding the close match for the movie name given by the user

In [20]:
close_match = difflib.get_close_matches(movie_name, names_of_movies)

In [21]:
close_match

['Alone in the Dark', 'Get on the Bus', 'Mondays in the Sun']

In [22]:
closest_match = close_match[0]
closest_match

'Alone in the Dark'

Find the index of the movie with title

In [23]:
index_of_movie = data[data.title == closest_match]['index'].values[0]

In [24]:
index_of_movie

2237

get the list of similar movies

In [25]:
similarty_score = list(enumerate(similarity[index_of_movie]))
similarty_score

[(0, 0.04980059829068201),
 (1, 0.06515433582823757),
 (2, 0.005755105827722747),
 (3, 0.043706739775795936),
 (4, 0.005477340238408214),
 (5, 0.029939090284235192),
 (6, 0.0),
 (7, 0.005677718771891041),
 (8, 0.019305751135690125),
 (9, 0.016627057567693684),
 (10, 0.02412056374540725),
 (11, 0.00965856242511165),
 (12, 0.07800101164985328),
 (13, 0.009411722798139037),
 (14, 0.038550386691804855),
 (15, 0.010727078131222966),
 (16, 0.005383684308126108),
 (17, 0.016468547236812514),
 (18, 0.009754286349388824),
 (19, 0.02903843582383122),
 (20, 0.028780968567897323),
 (21, 0.004720763362827673),
 (22, 0.05930941989004808),
 (23, 0.01764373049343221),
 (24, 0.020990595474452606),
 (25, 0.005283376664214817),
 (26, 0.005859899542846846),
 (27, 0.014080474684033532),
 (28, 0.07499797056515194),
 (29, 0.011375579421086409),
 (30, 0.03202194837883822),
 (31, 0.014538586470990666),
 (32, 0.02078965521429149),
 (33, 0.0109061985931598),
 (34, 0.03084161636636455),
 (35, 0.005923151786944678

In [26]:
len(similarty_score)

4803

Sorting the movies based on their similarity scores

In [27]:
sorted_similar_movies = sorted(similarty_score, key = lambda x:x[1], reverse = True)
sorted_similar_movies

[(2237, 1.0000000000000002),
 (784, 0.1759836872492178),
 (4608, 0.1546873917087874),
 (1939, 0.15277775661899234),
 (315, 0.14752630699946115),
 (3856, 0.12838526952171841),
 (2687, 0.1278108377805569),
 (2592, 0.12098583060176875),
 (4778, 0.1183648527875495),
 (2567, 0.11416256310780168),
 (1274, 0.10749410745654764),
 (901, 0.10565996826948648),
 (2300, 0.10251981066594323),
 (1265, 0.10185344380656836),
 (627, 0.10134847836306105),
 (759, 0.09914354360136035),
 (1722, 0.09816369292855077),
 (3172, 0.09680095676455194),
 (3843, 0.09623551906438035),
 (2762, 0.09324814744989376),
 (529, 0.09307928381571876),
 (1056, 0.09249969966940755),
 (3353, 0.09197135588673852),
 (1456, 0.09154841279360712),
 (3671, 0.0912020474675047),
 (1238, 0.09089245523965399),
 (3456, 0.08851550322494722),
 (4450, 0.08766587144180549),
 (3030, 0.08632468235374287),
 (4132, 0.08527137199918237),
 (2042, 0.08521341771128796),
 (1821, 0.08457115869548718),
 (217, 0.08356952156481398),
 (1914, 0.0811476642600

Print the name of the similar movies based on the index

In [28]:
print('Movies suggested for you: \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = data[data.index==index]['title'].values[0]

  if(i<31):
    print(i, '.', title_from_index)
    i+=1

Movies suggested for you: 

1 . Alone in the Dark
2 . In the Name of the King: A Dungeon Siege Tale
3 . The Prophecy
4 . BloodRayne
5 . The Mummy Returns
6 . In the Name of the King III
7 . My Boss's Daughter
8 . Highlander: Endgame
9 . Rampage
10 . Book of Shadows: Blair Witch 2
11 . Just Visiting
12 . Robin Hood: Prince of Thieves
13 . The Fall of the Roman Empire
14 . FearDotCom
15 . The Last Legion
16 . The Relic
17 . Dracula 2000
18 . The Contender
19 . Sharknado
20 . Serial Mom
21 . Tears of the Sun
22 . Blade
23 . The Descent
24 . Bound by Honor
25 . Yoga Hosers
26 . The Island of Dr. Moreau
27 . National Lampoon’s Van Wilder
28 . American Hero
29 . Tremors
30 . Eden Lake


Movie Recommendation system

In [29]:
movie_name = input('Enter your favourite name: ')
names_of_movies = data['title'].tolist()
close_match = difflib.get_close_matches(movie_name, names_of_movies)
closest_match = close_match[0]
index_of_movie = data[data.title == closest_match]['index'].values[0]
similarty_score = list(enumerate(similarity[index_of_movie]))
sorted_similar_movies = sorted(similarty_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you: \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = data[data.index==index]['title'].values[0]

  if(i<31):
    print(i, '.', title_from_index)
    i+=1

Enter your favourite name: american sniper
Movies suggested for you: 

1 . American Sniper
2 . J. Edgar
3 . The Hurt Locker
4 . The Messenger
5 . Space Cowboys
6 . Firefox
7 . Lone Survivor
8 . Absolute Power
9 . High Plains Drifter
10 . Burnt
11 . Act of Valor
12 . Pale Rider
13 . Gran Torino
14 . Red State
15 . Blood Work
16 . Gods and Generals
17 . Redacted
18 . The Bridges of Madison County
19 . Midnight in the Garden of Good and Evil
20 . A Bridge Too Far
21 . Jarhead
22 . Enemy at the Gates
23 . Unforgiven
24 . Jersey Boys
25 . Behind Enemy Lines
26 . Lucky Number Slevin
27 . Navy Seals vs. Zombies
28 . Joy
29 . Stop-Loss
30 . Under Siege 2: Dark Territory
